In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
import random
import warnings

warnings.filterwarnings('ignore')


categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]


# ============== 1) 데이터 로드 & 기본 전처리 예시 ==============
train = pd.read_csv('../data/train.csv').drop(columns=['ID'])
test = pd.read_csv('../data/test.csv').drop(columns=['ID'])

# 타겟 설정
y = train['임신 성공 여부']
X = train.drop(columns=['임신 성공 여부'])

# ============== 2) "절대 드롭 금지" 컬럼 설정 ==============
must_keep_cols = {
    "이식된 배아 수": None,
    "배아 이식률": None,
    "시술 당시 나이": None,
    "배아 이식 경과일": None,
    "총 임신 횟수": None,
    "총 출산 횟수": None,
    "수집된 신선 난자 수": None,
    "난자 출처": None,
    "저장된 배아 수": None,
    "배란 자극 여부": None,
    "임신 성공률": None,
    "시술 시기 코드": None,
    "난자 기증자 나이": None,
    "클리닉 내 총 시술 횟수": None,
    "IVF 시술 횟수": None,
    "미세주입 배아 이식률": None,
    "총 생성 배아 수": None,
    "불임 원인 - 남성 요인": None
}

Must_drop_columns = [
    '불임 원인 - 여성 요인',
    '불임 원인 - 자궁경부 문제',
    '불임 원인 - 정자 면역학적 요인',
    '불임 원인 - 정자 운동성',
    '불임 원인 - 정자 형태',
    '난자 채취 경과일',
    '난자 해동 경과일'
]

all_columns = [
    "시술 시기 코드", "시술 당시 나이", "시술 유형", "특정 시술 유형", "배란 자극 여부", "배란 유도 유형", "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부", "착상 전 유전 진단 사용 여부", "남성 주 불임 원인", "남성 부 불임 원인", "여성 주 불임 원인",
    "여성 부 불임 원인", "부부 주 불임 원인", "부부 부 불임 원인", "불명확 불임 원인", "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인", "불임 원인 - 배란 장애", "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증", "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태", "배아 생성 주요 이유", "총 시술 횟수", "클리닉 내 총 시술 횟수", "IVF 시술 횟수",
    "DI 시술 횟수", "총 임신 횟수", "IVF 임신 횟수", "DI 임신 횟수", "총 출산 횟수", "IVF 출산 횟수", "DI 출산 횟수",
    "난자 출처", "정자 출처", "난자 기증자 나이", "정자 기증자 나이", "동결 배아 사용 여부", "신선 배아 사용 여부",
    "기증 배아 사용 여부", "대리모 여부", "PGD 시술 여부", "PGS 시술 여부", "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수", "미세주입된 난자 수", "미세주입에서 생성된 배아 수", "이식된 배아 수", "미세주입 배아 이식 수",
    "저장된 배아 수", "미세주입 후 저장된 배아 수", "해동된 배아 수", "해동 난자 수", "수집된 신선 난자 수", "저장된 신선 난자 수",
    "혼합된 난자 수", "파트너 정자와 혼합된 난자 수", "기증자 정자와 혼합된 난자 수", "난자 채취 경과일", "난자 해동 경과일",
    "난자 혼합 경과일", "배아 이식 경과일", "배아 해동 경과일"
]

# 추가적으로 드롭할 수 있는 후보(= 랜덤 서치 대상) 설정
#   - must_keep_cols에 없고, Must_drop_columns에도 없는 것들만이 대상
additional_drop_candidates = [
    col for col in all_columns if col not in must_keep_cols and col not in Must_drop_columns
]

# 1) Must_drop_columns를 무조건 드롭
X.drop(columns=Must_drop_columns, errors='ignore', inplace=True)
test.drop(columns=Must_drop_columns, errors='ignore', inplace=True)

# 그 뒤 다시 컬럼 집합 업데이트
train_cols = set(X.columns)
test_cols = set(test.columns)

# 실제로 train/test에 존재하는 컬럼 중, must_keep이 아닌 것만 후보로 필터링
filtered_candidates = [
    c for c in additional_drop_candidates
    if c in train_cols and c in test_cols
]
print("최종 실험 대상 후보 컬럼:", filtered_candidates)


# (결측치 처리 예시)
for col in numeric_columns:
    if col in X.columns:
        mode_value = X[col].mode()[0]
        X[col].fillna(mode_value, inplace=True)
    if col in test.columns:
        test[col].fillna(mode_value, inplace=True)

for col in categorical_columns:
    if col in X.columns:
        X[col] = X[col].astype(str)
    if col in test.columns:
        test[col] = test[col].astype(str)

# ============== 5) Random Search로 부분집합 반복 시도 ==============
N_EXPERIMENTS = 20  # 시도할 횟수
best_auc = 0.0
best_subset = []  # 지금까지 최고의 점수를 낸, 드롭할 컬럼들

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=17)
random.seed(42)

for exp_num in range(N_EXPERIMENTS):
    # 1) 랜덤으로 부분집합(드롭할 컬럼들) 생성
    subset_size = random.randint(0, len(filtered_candidates))  
    drop_subset = random.sample(filtered_candidates, subset_size)
    
    # 2) 해당 부분집합을 드롭한 복사본 데이터
    X_dropped = X.drop(columns=drop_subset, errors='ignore')
    test_dropped = test.drop(columns=drop_subset, errors='ignore')
    
    # 3) 모델 학습 후 K-Fold AUC 계산
    scores = []
    for train_idx, val_idx in cv.split(X_dropped, y):
        X_train, X_val = X_dropped.iloc[train_idx], X_dropped.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # "시술 당시 나이"가 "알 수 없음"이면 제거 (예시)
        if '시술 당시 나이' in X_train.columns:
            mask = X_train['시술 당시 나이'] != "알 수 없음"
            X_train = X_train[mask]
            y_train = y_train[mask]
        
        cat_cols_in_use = [c for c in categorical_columns if c in X_train.columns]
        
        model = CatBoostClassifier(
            auto_class_weights='Balanced',
            eval_metric='AUC',
            random_state=17,
            task_type="GPU",  # 필요한 환경에 따라 CPU/GPU로 수정
            devices='0',
            verbose=False
        )
        model.fit(
            X_train, y_train,
            cat_features=cat_cols_in_use,
            early_stopping_rounds=100,
            eval_set=(X_val, y_val)
        )
        val_auc = model.get_best_score()['validation']['AUC']
        scores.append(val_auc)
    
    mean_auc = np.mean(scores)
    
    # 4) AUC 갱신 여부 확인
    if mean_auc > best_auc:
        best_auc = mean_auc
        best_subset = drop_subset
        print(f"[{exp_num+1}/{N_EXPERIMENTS}] AUC 개선! {best_auc:.4f}, 드롭 컬럼: {best_subset}")
    else:
        print(f"[{exp_num+1}/{N_EXPERIMENTS}] AUC={mean_auc:.4f} (현재 최고: {best_auc:.4f})")

# ============== 6) 최종 결과 ==============
print("\n===== Random Search 종료 =====")
print(f"최고 AUC: {best_auc:.4f}")
print("드롭하면 좋은 컬럼들:", best_subset)

# ============== 7) 최종 모델 학습 & 제출 (예시) ==============
X_final = X.drop(columns=best_subset, errors='ignore')
test_final = test.drop(columns=best_subset, errors='ignore')

# model_final = CatBoostClassifier(
#     auto_class_weights='Balanced',
#     eval_metric='AUC',
#     random_state=17,
#     task_type="GPU", 
#     devices='0'
# )
# model_final.fit(
#     X_final, y,
#     cat_features=[c for c in categorical_columns if c in X_final.columns],
#     verbose=100
# )
# test_pred = model_final.predict_proba(test_final)[:,1]

# sample_submission = pd.read_csv('../data/sample_submission.csv')
# sample_submission['probability'] = test_pred
# sample_submission.to_csv('./catboost_random_search_best_drop.csv', index=False)


최종 실험 대상 후보 컬럼: ['시술 유형', '특정 시술 유형', '배란 유도 유형', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 배란 장애', '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '배아 생성 주요 이유', '총 시술 횟수', 'DI 시술 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '정자 출처', '정자 기증자 나이', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부', '임신 시도 또는 마지막 임신 경과 연수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '미세주입 배아 이식 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '난자 혼합 경과일', '배아 해동 경과일']


KeyboardInterrupt: 

In [ ]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold

# -----------------------------
# 1) KFold 설정
# -----------------------------
n_splits = 10
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=17)

# '시술 당시 나이'가 '알 수 없음'인 행 제거 로직이 필요하다면 여기에 추가 가능
# 예시:
# mask = (X_final['시술 당시 나이'] != "알 수 없음")
# X_final = X_final[mask]
# y = y[mask]

# -----------------------------
# 2) KFold를 이용해 각 Fold별 학습 & 모델 저장
# -----------------------------
test_preds = np.zeros((len(test_final), n_splits))  # (테스트셋 행 수, n_splits)

for fold_idx, (train_idx, val_idx) in enumerate(cv.split(X_final, y)):
    print(f"===== Fold {fold_idx+1}/{n_splits} =====")
    
    X_train = X_final.iloc[train_idx]
    y_train = y.iloc[train_idx]
    X_val = X_final.iloc[val_idx]
    y_val = y.iloc[val_idx]
    
    # 만약 다시 한 번 '시술 당시 나이' == "알 수 없음" 제거 로직이 필요하다면 각 fold 내에서 추가:
    # (필요 없다면 아래 주석 처리)
    # train_mask = (X_train['시술 당시 나이'] != "알 수 없음")
    # X_train = X_train[train_mask]
    # y_train = y_train[train_mask]
    
    cat_features_in_use = [c for c in X_train.columns if c in categorical_columns]
    
    model = CatBoostClassifier(
        auto_class_weights='Balanced',
        eval_metric='AUC',
        random_state=17,
        task_type="GPU",  # GPU 사용 환경이 아니면 "CPU"로 변경
        devices='0',      # GPU 번호
        verbose=100
    )
    
    model.fit(
        X_train, y_train,
        cat_features=cat_features_in_use,
        eval_set=(X_val, y_val),
        early_stopping_rounds=100
    )
    
    # 2-1) 모델 파일(.cbm)로 저장
    model.save_model(f"./catboost_fold_{fold_idx}.cbm")
    
    # 2-2) Test 데이터 예측(확률) 저장
    fold_test_pred = model.predict_proba(test_final)[:, 1]
    test_preds[:, fold_idx] = fold_test_pred

# -----------------------------
# 3) KFold 앙상블(평균) 결과로 최종 제출 파일 생성
# -----------------------------
test_pred_ensemble = test_preds.mean(axis=1)

sample_submission = pd.read_csv('../data/sample_submission.csv')
sample_submission['probability'] = test_pred_ensemble
sample_submission.to_csv('./catboost_kfold_ensemble_submission.csv', index=False)